In [1]:
import pandas as pd
import numpy as np
import datetime
from math import floor, ceil

## Data Description
#### Data: Transactions
* **TransactionID**: The unique transaction code.
* **ChargePointID**: The unique point of the charging station.
* **ConnectorID**: Indicates what connector was used for the transactions.
* **UTCTransactionStart**: The moment the transaction was started (logged in locale time zone)
* **UTCTransactionStop**: The moment the plug was disconnected and the transaction was stopped
* **StartCard**: The RFID card (hashed) which has been used to start a transaction
* **ConnectedTime**: Total time wherein energy transfer took place.
* **ChargeTime**: Total time wherein energy transfer took place
* **TotalEnergy**: The total energy demand (kWh) per session
* **MaxPower**: The maximum charging rate (kW) during a session

In [2]:
# Parameters
minute_delta = 15
max_capacity = 80

In [3]:
df_eld = pd.read_csv("data/elaadnl_open_ev_datasets.csv", parse_dates = ["UTCTransactionStart", "UTCTransactionStop"]).rename(columns={"UTCTransactionStart": "TransactionStartDT",
                                                                                                                                       "UTCTransactionStop": "TransactionStopDT"})
df_eld.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   TransactionId       10000 non-null  int64         
 1   ChargePoint         10000 non-null  object        
 2   Connector           10000 non-null  int64         
 3   TransactionStartDT  10000 non-null  datetime64[ns]
 4   TransactionStopDT   10000 non-null  datetime64[ns]
 5   StartCard           10000 non-null  object        
 6   ConnectedTime       10000 non-null  float64       
 7   ChargeTime          10000 non-null  float64       
 8   TotalEnergy         10000 non-null  float64       
 9   MaxPower            10000 non-null  float64       
dtypes: datetime64[ns](2), float64(4), int64(2), object(2)
memory usage: 781.4+ KB


In [4]:
df_eld["BatteryCapacity"] = max_capacity
df_eld["TotalEnergy"] = df_eld[["TotalEnergy", "BatteryCapacity"]].min(axis=1) 
df_eld["SOC_arr"] = df_eld["TotalEnergy"] / df_eld["BatteryCapacity"]
df_eld["TotalEnergy"].describe()

count    10000.000000
mean        13.625519
std         14.234003
min          0.100000
25%          4.530000
50%          7.830000
75%         18.262500
max         80.000000
Name: TotalEnergy, dtype: float64

In [5]:
start_dt = df_eld["TransactionStartDT"].min()
end_dt = df_eld["TransactionStartDT"].max()
print(f"{start_dt=}, {type(start_dt)=}")
print(f"{end_dt=}, {type(end_dt)=}")
diff_dt = end_dt - start_dt
print(f"{diff_dt=}, {type(diff_dt)=}")

start_dt=Timestamp('2019-01-01 00:30:00'), type(start_dt)=<class 'pandas._libs.tslibs.timestamps.Timestamp'>
end_dt=Timestamp('2019-12-31 21:22:00'), type(end_dt)=<class 'pandas._libs.tslibs.timestamps.Timestamp'>
diff_dt=Timedelta('364 days 20:52:00'), type(diff_dt)=<class 'pandas._libs.tslibs.timedeltas.Timedelta'>


In [6]:
df_eld["TransactionStartDate"] = df_eld["TransactionStartDT"].apply(lambda x: x.date())
df_eld["TransactionStopDate"] = df_eld["TransactionStopDT"].apply(lambda x: x.date())

df_eld["TransactionStartSecDay"] = df_eld["TransactionStartDT"].apply(lambda x: datetime.timedelta(hours=x.hour,minutes=x.minute,seconds=x.second).total_seconds())
df_eld["TransactionStopSecDay"] = df_eld["TransactionStopDT"].apply(lambda x: datetime.timedelta(hours=x.hour,minutes=x.minute,seconds=x.second).total_seconds())

df_eld["TransactionStartHourDay"] = df_eld["TransactionStartSecDay"] // (60*60)
df_eld["TransactionStopHourDay"] = df_eld["TransactionStopSecDay"] // (60*60)

df_eld["TransactionStartAbsSec"] = (df_eld["TransactionStartDT"] - start_dt).dt.total_seconds()
df_eld["TransactionStopAbsSec"] = (df_eld["TransactionStopDT"] - start_dt).dt.total_seconds()

df_eld["TransactionStartTS"] = df_eld["TransactionStartAbsSec"] // (60 * minute_delta) + 1
df_eld["TransactionStopTS"]  = df_eld["TransactionStopAbsSec"]  // (60 * minute_delta) + 2


In [7]:
df_eld["ConnectedTimeMin"] = df_eld["ConnectedTime"]*60
df_eld["measConnectedTimeMin"] = (df_eld["TransactionStopAbsSec"] - df_eld["TransactionStartAbsSec"]) / 60
df_eld["errConnectedTime"] = df_eld["ConnectedTimeMin"] - df_eld["measConnectedTimeMin"]
df_eld[["ConnectedTimeMin", "measConnectedTimeMin", "errConnectedTime"]].describe()

,ConnectedTimeMin,measConnectedTimeMin,errConnectedTime
count,10000.000000,10000.000000,10000.000000
mean,349.364100,349.370400,-0.006300
std,425.781684,425.784333,0.443847
min,1.200000,1.000000,-1.200000
25%,90.600000,91.000000,-0.400000
50%,202.800000,203.000000,0.000000
75%,504.600000,504.000000,0.400000
max,9670.200000,9670.000000,1.200000


In [8]:
df_eld[["TransactionStartTS", "TransactionStopTS"]].describe()

,TransactionStartTS,TransactionStopTS
count,10000.000000,10000.000000
mean,18384.218500,18408.510600
std,10611.352311,10612.515049
min,1.000000,33.000000
25%,8977.500000,8993.750000
50%,18272.000000,18293.500000
75%,28250.250000,28276.000000
max,35028.000000,35104.000000


In [9]:
df_eld = df_eld.sort_values("TransactionStartTS").reset_index(drop = True)
df_eld.head()

,TransactionId,ChargePoint,Connector,TransactionStartDT,TransactionStopDT,StartCard,ConnectedTime,ChargeTime,TotalEnergy,MaxPower,...,TransactionStopSecDay,TransactionStartHourDay,TransactionStopHourDay,TransactionStartAbsSec,TransactionStopAbsSec,TransactionStartTS,TransactionStopTS,ConnectedTimeMin,measConnectedTimeMin,errConnectedTime
0,3261657,e62c50d1be0a2f80ec51d471f9630a4e,2,2019-01-01 00:30:00,2019-01-01 08:24:00,826d337c1d84900452edfcb27a3fb11bcbd06f35da8406...,7.91,1.00,6.53,9.818,...,30240.0,0.0,8.0,0.0,28440.0,1.0,33.0,474.6,474.0,0.6
1,3262038,9bae10789a789973cc7f05d2a96df76f,1,2019-01-01 10:36:00,2019-01-01 12:52:00,26612ba143072faddaa884c2b04208992c5f44d97e5a24...,2.27,2.27,8.08,3.600,...,46320.0,10.0,12.0,36360.0,44520.0,41.0,51.0,136.2,136.0,0.2
2,3262129,5ab468315a1f42feb6d0a87307593352,1,2019-01-01 11:49:00,2019-01-01 17:50:00,8478920deafe847713d1707d8f71a043c59fa066e55020...,6.02,3.75,11.56,3.400,...,64200.0,11.0,17.0,40740.0,62400.0,46.0,71.0,361.2,361.0,0.2
3,3262170,00fa448dfcba79a12a3cea56fb48d63f,2,2019-01-01 12:22:00,2019-01-01 14:29:00,e8e1953d3e6221f971ce3299d2fa8275cdc9e7700e2b39...,2.12,0.87,3.84,7.973,...,52140.0,12.0,14.0,42720.0,50340.0,48.0,57.0,127.2,127.0,0.2
4,3262208,e35645c6b4adb10224ecfbdff6bc5473,1,2019-01-01 12:51:00,2019-01-01 15:16:00,6490d53fbbe95d74592928292858f9b156d838f08d062e...,2.41,2.41,8.50,3.592,...,54960.0,12.0,15.0,44460.0,53160.0,50.0,61.0,144.6,145.0,-0.4


In [10]:
df_eld.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 25 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   TransactionId            10000 non-null  int64         
 1   ChargePoint              10000 non-null  object        
 2   Connector                10000 non-null  int64         
 3   TransactionStartDT       10000 non-null  datetime64[ns]
 4   TransactionStopDT        10000 non-null  datetime64[ns]
 5   StartCard                10000 non-null  object        
 6   ConnectedTime            10000 non-null  float64       
 7   ChargeTime               10000 non-null  float64       
 8   TotalEnergy              10000 non-null  float64       
 9   MaxPower                 10000 non-null  float64       
 10  BatteryCapacity          10000 non-null  int64         
 11  SOC_arr                  10000 non-null  float64       
 12  TransactionStartDate     10000 no

In [11]:
df_eld.describe()

,TransactionId,Connector,ConnectedTime,ChargeTime,TotalEnergy,MaxPower,BatteryCapacity,SOC_arr,TransactionStartSecDay,TransactionStopSecDay,TransactionStartHourDay,TransactionStopHourDay,TransactionStartAbsSec,TransactionStopAbsSec,TransactionStartTS,TransactionStopTS,ConnectedTimeMin,measConnectedTimeMin,errConnectedTime
count,1.000000e+04,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.0,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000,1.000000e+04,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,3.452817e+06,1.360300,5.822735,2.87803,13.625519,5.589366,80.0,0.170319,46479.42600,48200.370000,12.412600,12.902500,1.654532e+07,1.656628e+07,18384.218500,18408.510600,349.364100,349.370400,-0.006300
std,1.076139e+05,0.480111,7.096361,2.64428,14.234003,3.333663,0.0,0.177925,16276.03407,16136.440174,4.532131,4.495377,9.550218e+06,9.551261e+06,10611.352311,10612.515049,425.781684,425.784333,0.443847
min,3.261657e+06,1.000000,0.020000,0.02000,0.100000,0.167000,80.0,0.001250,120.00000,360.000000,0.000000,0.000000,0.000000e+00,2.844000e+04,1.000000,33.000000,1.200000,1.000000,-1.200000
25%,3.360979e+06,1.000000,1.510000,1.25000,4.530000,3.398000,80.0,0.056625,32460.00000,36420.000000,9.000000,10.000000,8.079225e+06,8.092980e+06,8977.500000,8993.750000,90.600000,91.000000,-0.400000
50%,3.452182e+06,1.000000,3.380000,2.24000,7.830000,3.640000,80.0,0.097875,46140.00000,48720.000000,12.000000,13.000000,1.644426e+07,1.646289e+07,18272.000000,18293.500000,202.800000,203.000000,0.000000
75%,3.547667e+06,2.000000,8.410000,3.50000,18.262500,7.277750,80.0,0.228281,59580.00000,59520.000000,16.000000,16.000000,2.542464e+07,2.544678e+07,28250.250000,28276.000000,504.600000,504.000000,0.400000
max,3.634120e+06,2.000000,161.170000,25.67000,80.000000,22.500000,80.0,1.000000,86340.00000,86340.000000,23.000000,23.000000,3.152472e+07,3.159180e+07,35028.000000,35104.000000,9670.200000,9670.000000,1.200000


In [12]:
df_eld.to_csv("data/prepared_elaad_transactions.csv", index=False)

In [13]:
print(df_eld["ChargePoint"].nunique())
print(df_eld["Connector"].nunique())
print((df_eld["ChargePoint"] + df_eld["Connector"].astype(str)).nunique())


850
2
1293
